## A (very) brief introduction to Color Doppler 

Color doppler is a medical imaging modality for blood flow imaging.  
It bases on the Doppler effect.  
Doppler effect is a shift of a received signal frequency, when the wave source is moving relatively to the receiver.  
In medicine 'sources' (i.e. moving tissue, specifically blood) do not emits acoustic waves, but are 'iluminated' by a ultrasound pulses produced by a probe.  
It can be shown [Evans2000] that the shift in the received frequency is given by following equation:

$
f_d = f_t - f_r = \frac{2f_tv\cos{\alpha}}{c}
$

where
* $f_d$ - frequency shift, or doppler frequency,
* $f_t$ - transmitted frequency, 
* $f_r$ - received frequency, 
* $v$ - speed of the blood,
* $\alpha$ - the angle between the ultrasound beam and the direction of motion of the blood,
* $c$ - speed of sound in the medium. It is usually assumed that in soft tissue $c = 1540 [m/s]$.



<!-- When the medical probe transmits the ultrasound pulse, and it is scattered on moving blood (i.e. on blood cells), the received echoes changes in phase.    -->
In classical approach the probe transmits a series of (quite long) ultrasound pulses, receiving echoes after each transmit.  
The series consists of $N$ transmit/receive (TR) events, and the higher $N$, the higher sensitivity, but lower doppler framerate.  
Typically $N$ could be in the range of $8-16$ for classical methods and $32-256$ for synthetic aperture methods, however there are no strict rules.  
The TR events in the series are repeated with constant Pulse Repetition Frequency (PRF).  
Thus the time between TR events - Pulse Repetition Interval (PRI) is equal $\frac{1}{PRF}$.  
The received signals are IQ demodulated. The $f_d$ can be estimated from IQ signal by means of autocorrelation esitmator:  

$
\overline{f_d} = \frac{1}{2\pi{}PRI} 
    \tan^{-1}{\left\{ 
        \frac{\sum^{N}_{i=1}{Q(i)I(i-1) - I(i)Q(i-1)}}
             {\sum^{N}_{i=1}{I(i)I(i-1) + Q(i)Q(i-1)}}
    \right\}}
$



<!-- , and next the phase is estimated for each sample.  
Then, for each sample the phase changes $\Delta{\theta}$ from TR to TR are calculated. Sometimes it is refered as phase changes in 'slow time'.  
The doppler frequency (averaged over time) can be calculated from the formula

$
\overline{f_d} = \frac{1}{N-1} \sum_{n=1}^{N-1} \frac{\Delta{\theta}_{n}}{PRI}
$

 -->
Then, we can use the following formula to estimate (average) speed of the blood flow.  

$
v_s = \frac{\overline{f_d}}{f_t} \frac{c}{2\cos{\alpha}}
$





## The complete processing for Color Doppler using CUDA

Data loading

In [9]:
from scipy.io import loadmat
import numpy as np

data = loadmat('./data/iqImgML_batch01.mat')
print(data.keys())

c = np.squeeze(data['sos']) # speed of sound
iq


# decimation - the fs=1e8 is not necessary high
dec = 4
rf = signal.decimate(rf, dec)
fs = fs/dec

print(f'Data loaded. Sampling frequency [MHz]: {fs*1e-6}')

dict_keys(['__header__', '__version__', '__globals__', 'iq', 'prf', 'sos', 'txFrequency', 'xGrid', 'zGrid'])


KeyError: 'rf'

Kernel definitions

In [5]:
import cupy as cp

source = r"""

#include <cupy/complex.cuh>
extern "C" __global__ 
void dopplerColor(float *color, 
                  const complex<float> *iqFrames, 
                  const int nz, 
                  const int nx, 
                  const int nFrames)
                  
{
    int z = blockIdx.x * blockDim.x + threadIdx.x;
    int x = blockIdx.y * blockDim.y + threadIdx.y;
    
    complex<float> iqCurrent, iqPrevious;
    complex<float> auxColor(0.0f, 0.0f);
    float ic, qc, ip, qp, nom, den;
    
    if (z>=nz || x>=nx) {
        return;
    }
    
    /* Color estimation */
    iqCurrent = iqFrames[z + x * nz];
    for (int iFrame=1; iFrame<nFrames; iFrame++) {
        iqPrevious = iqCurrent;
        iqCurrent = iqFrames[z + x*nz + iFrame*nz*nx];
        ic = real(iqCurrent);
        qc = imag(iqCurrent);
        ip = real(iqPrevious);
        qp = imag(iqPrevious);
        den += ic*ip + qc*qp;
        nom += qc*ip - ic*qp;
    }
    color[z + x*nz] = atan2f(nom, den);
}

"""
dopplerColor = cp.RawKernel(source, 'dopplerColor')

# kernel test
nx = 8
nz = 8
nFrames = 8
# iq = cp.arange(nx * nz * nFrames, dtype=cp.complex64).reshape(nz, nx, nFrames)
iq = cp.random.random(nx * nz * nFrames) + cp.random.random(nx * nz * nFrames) * 1j
iq = iq.reshape(nz, nx, nFrames)
color =  cp.zeros((nz, nx), dtype=cp.complex64)
power =  cp.zeros((nz, nx), dtype=cp.complex64)
grid = (nx, nz)
block = (nFrames, )
# print(iq)
%time dopplerColor(grid, block, (color, iq, nz, nx, nFrames))
print(color)


CPU times: user 262 µs, sys: 0 ns, total: 262 µs
Wall time: 140 µs
[[           nan+1.2350037e+00j -3.1415927e+00+1.5707951e+00j
   3.1415927e+00-1.5707964e+00j  3.1415927e+00-6.9826434e-04j
   7.2909734e-06-1.5707964e+00j  1.5707964e+00+3.1415927e+00j
   3.1415927e+00          +nanj            nan+0.0000000e+00j]
 [ 7.6189742e-11          +nanj  0.0000000e+00+3.1415927e+00j
             nan+0.0000000e+00j  3.1415927e+00+3.1415927e+00j
   0.0000000e+00-1.1411368e-01j  1.5707964e+00+1.5707964e+00j
   3.1415915e+00+0.0000000e+00j -1.5557959e+00-3.1415927e+00j]
 [ 1.5707964e+00-3.1415927e+00j            nan-0.0000000e+00j
  -3.1438464e-07+6.2227885e-09j            nan+3.7078307e-10j
   3.1415927e+00-1.5707964e+00j            nan+0.0000000e+00j
  -1.9088589e-01+1.5707964e+00j            nan-5.4592493e-07j]
 [           nan          +nanj -0.0000000e+00+3.1415927e+00j
  -2.1036710e-08-3.1415896e+00j -0.0000000e+00          +nanj
             nan-3.1415927e+00j -3.1415927e+00-1.4982168e-11j


## A very brief introduction to Vector Doppler 

## The complete processing for Vector Doppler using CUDA

## Doing B-mode and Color/Vector processing in the same time - a single GPU (multiple streams) example

### References

[Evans2000] Evans, David H., and W. Norman McDicken. Doppler ultrasound: physics, instrumentation and signal processing. Wiley-Blackwell, 2000.